In [57]:
import pandas as pd
import numpy as np
import mpu
import json
import re  
pd.set_option('display.max_columns', None)
house = pd.read_csv('../data/house_combined.csv')
bus = pd.read_csv('../data/bus_stops_file.csv')
train = pd.read_csv('../data/train_stops_file.csv')
tram = pd.read_csv('../data/tram_stops_file.csv')
vline = pd.read_csv('../data/vline_stops_file.csv')
population = pd.read_excel('../data/Population.xlsx')
income = pd.read_excel('../data/Income.xlsx',sheet_name="Forecasted Income",header=1)
male = pd.read_excel('../data/Sex & Age by Year.xlsx', sheet_name="Male")
female = pd.read_excel('../data/Sex & Age by Year.xlsx', sheet_name="Female")
school = pd.read_csv('../data/school.csv',sep=",", encoding='cp1252')
hospital = pd.read_csv('../data/hospital_csv',sep=",", encoding='cp1252')
data_2019 = pd.read_csv('../data/2019_data.csv',sep=",", encoding='cp1252')
data_2020 = pd.read_csv('../data/2020_data.csv',sep=",", encoding='cp1252')
data_2021 = pd.read_csv('../data/2021_data.csv',sep=",", encoding='cp1252')
data_2022 = pd.read_csv('../data/2022_data.csv',sep=",", encoding='cp1252')


In [58]:
data_2019[data_2019["postcode"]>3000]
house.columns

Index(['Unnamed: 0', 'propertyTypes', 'objective', 'addressParts',
       'advertiserIdentifiers', 'bathrooms', 'bedrooms', 'carspaces',
       'dateUpdated', 'dateListed', 'geoLocation', 'priceDetails',
       'landAreaSqm', 'dateAvailable', 'postcode', 'surburb', 'year', 'month',
       'price', 'bond', 'rent', 'state'],
      dtype='object')

In [60]:

need_col = [
    "index",
    "propertyTypes",
    "bathrooms",
    "bedrooms",
    "carspaces",
    "dateUpdated",
    "dateListed",
    "geoLocation",
    "landAreaSqm",
    "postcode",
    "surburb",
    'year', 
    'month',
    'price', 
    'bond',
    'rent'
]
init_house = house[need_col]

init_house["geoLocation"] = init_house["geoLocation"].apply(lambda x: re.sub('\'','\"', str(x)))
init_house = init_house[init_house["geoLocation"] != 'nan']
init_house['longitude'] = init_house['geoLocation'].apply(lambda d: json.loads(d)["longitude"])
init_house['latitude'] = init_house['geoLocation'].apply(lambda d: json.loads(d)["latitude"])

KeyError: "['index'] not in index"

In [ ]:

merge_bus = pd.merge(init_house,bus,how='left',on='postcode')
merge_bus["tmp"] = np.sin(merge_bus["stop_latitude"] - merge_bus["latitude"] / 2)**2 + np.cos(merge_bus["latitude"]) * np.cos(merge_bus["stop_latitude"]) * np.sin(merge_bus["stop_longitude"] - merge_bus["longitude"] / 2)**2
merge_bus["bus_stop_distance"] = 2 * np.arctan2(np.sqrt(merge_bus["tmp"]), np.sqrt(1 - merge_bus["tmp"]))


c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
bus_stop_min = pd.DataFrame(merge_bus.groupby(["index",'postcode'])["bus_stop_distance"].min())
# bus_stop_min
bus_stop_count = pd.DataFrame(merge_bus.groupby(["index",'postcode'])["bus_stop_distance"].count())
# bus_stop_count
init_house = pd.merge(init_house,bus_stop_min,how='left',left_on=['index','postcode'],right_on=['index','postcode'])
init_house = pd.merge(init_house,bus_stop_count,how='left',left_on=['index','postcode'],right_on=['index','postcode'])
init_house.rename(columns={"bus_stop_distance_x":"distance of the closest bus stop","bus_stop_distance_y":"count of bus stop"}, inplace=True)
# init_house

In [ ]:
merge_train = pd.merge(init_house,train,how='left',on='postcode')
merge_train["tmp"] = np.sin(merge_train["stop_latitude"] - merge_train["latitude"] / 2)**2 + np.cos(merge_train["latitude"]) * np.cos(merge_train["stop_latitude"]) * np.sin(merge_train["stop_longitude"] - merge_bus["longitude"] / 2)**2
merge_train["train_stop_distance"] = 2 * np.arctan2(np.sqrt(merge_train["tmp"]), np.sqrt(1 - merge_train["tmp"]))
train_stop_min = pd.DataFrame(merge_train.groupby(["index",'postcode'])["train_stop_distance"].min())
train_stop_count = pd.DataFrame(merge_train.groupby(["index",'postcode'])["train_stop_distance"].count())
init_house = pd.merge(init_house,train_stop_min,how='left',left_on=['index','postcode'],right_on=['index','postcode'])
init_house = pd.merge(init_house,train_stop_count,how='left',left_on=['index','postcode'],right_on=['index','postcode'])
init_house.rename(columns={"train_stop_distance_x":"distance of the closest train stop","train_stop_distance_y":"count of train stop"}, inplace=True)


In [ ]:
merge_tram = pd.merge(init_house,tram,how='left',on='postcode')
merge_tram["tmp"] = np.sin(merge_tram["stop_latitude"] - merge_tram["latitude"] / 2)**2 + np.cos(merge_tram["latitude"]) * np.cos(merge_tram["stop_latitude"]) * np.sin(merge_tram["stop_longitude"] - merge_bus["longitude"] / 2)**2
merge_tram["tram_stop_distance"] = 2 * np.arctan2(np.sqrt(merge_tram["tmp"]), np.sqrt(1 - merge_tram["tmp"]))
tram_stop_min = pd.DataFrame(merge_tram.groupby(["index",'postcode'])["tram_stop_distance"].min())
tram_stop_count = pd.DataFrame(merge_tram.groupby(["index",'postcode'])["tram_stop_distance"].count())
init_house = pd.merge(init_house,tram_stop_min,how='left',left_on=['index','postcode'],right_on=['index','postcode'])
init_house = pd.merge(init_house,tram_stop_count,how='left',left_on=['index','postcode'],right_on=['index','postcode'])
init_house.rename(columns={"tram_stop_distance_x":"distance of the closest tram stop","tram_stop_distance_y":"count of tram stop"}, inplace=True)

In [ ]:
merge_vline = pd.merge(init_house,vline,how='left',on='postcode')
merge_vline["tmp"] = np.sin(merge_vline["stop_latitude"] - merge_vline["latitude"] / 2)**2 + np.cos(merge_vline["latitude"]) * np.cos(merge_vline["stop_latitude"]) * np.sin(merge_vline["stop_longitude"] - merge_vline["longitude"] / 2)**2
merge_vline["vline_stop_distance"] = 2 * np.arctan2(np.sqrt(merge_vline["tmp"]), np.sqrt(1 - merge_vline["tmp"]))
vline_stop_min = pd.DataFrame(merge_vline.groupby(["index",'postcode'])["vline_stop_distance"].min())
vline_stop_count = pd.DataFrame(merge_vline.groupby(["index",'postcode'])["vline_stop_distance"].count())
init_house = pd.merge(init_house,vline_stop_min,how='left',left_on=['index','postcode'],right_on=['index','postcode'])
init_house = pd.merge(init_house,vline_stop_count,how='left',left_on=['index','postcode'],right_on=['index','postcode'])
init_house.rename(columns={"vline_stop_distance_x":"distance of the closest vline stop","vline_stop_distance_y":"count of vline stop"}, inplace=True)

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
merge_school = pd.merge(init_house,school,how='left',left_on='postcode',right_on="Address_Postcode")
merge_school["tmp"] = np.sin(merge_school["Y"] - merge_school["latitude"] / 2)**2 + np.cos(merge_school["latitude"]) * np.cos(merge_school["Y"]) * np.sin(merge_school["X"] - merge_vline["longitude"] / 2)**2
merge_school["school_distance"] = 2 * np.arctan2(np.sqrt(merge_school["tmp"]), np.sqrt(1 - merge_school["tmp"]))
school_stop_min = pd.DataFrame(merge_school.groupby(["index",'Address_Postcode'])["school_distance"].min())
school_stop_count = pd.DataFrame(merge_school.groupby(["index",'Address_Postcode'])["school_distance"].count())
init_house = pd.merge(init_house,school_stop_min,how='left',left_on=['index','postcode'],right_on=['index','Address_Postcode'])
init_house = pd.merge(init_house,school_stop_count,how='left',left_on=['index','postcode'],right_on=['index','Address_Postcode'])
init_house.rename(columns={"school_distance_x":"distance of the closest school","school_distance_y":"count of school"}, inplace=True)

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
merge_hospital = pd.merge(init_house,hospital,how='left',on='postcode')
merge_hospital
hospital_count = pd.DataFrame(merge_hospital.groupby(["index",'postcode'])["name"].count())
hospital_count
init_house = pd.merge(init_house,hospital_count,how='left',left_on=['index','postcode'],right_on=['index','postcode'])
init_house.rename(columns={"name":"count of hospital"}, inplace=True)

,index,propertyTypes,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,geoLocation,landAreaSqm,postcode,surburb,year,month,price,bond,rent,longitude,latitude,distance of the closest bus stop,count of bus stop,distance of the closest train stop,count of train stop,distance of the closest tram stop,count of tram stop,distance of the closest vline stop,count of vline stop,distance of the closest school,count of school,name
0,4256,['apartmentUnitFlat'],1.0,1.0,0.0,2019-01-09T00:23:18.123Z,2016-02-16T04:33:49Z,"{""latitude"": -37.8341628, ""longitude"": 144.960...",NaN,3205,south melbourne,2019,1,$360.00 per week,NaN,360.0,144.960873,-37.834163,0.436505,10,NaN,0,0.438447,24,NaN,0,0.441059,3.0,0
1,5429,['apartmentUnitFlat'],2.0,4.0,4.0,2019-01-03T22:51:13.063Z,2018-10-02T00:00:38Z,"{""latitude"": -37.8443239, ""longitude"": 144.943...",NaN,3207,port melbourne,2019,1,$1800.00 per week,NaN,1800.0,144.943722,-37.844324,0.360502,69,NaN,0,0.427958,3,NaN,0,0.413926,2.0,0
2,5437,['apartmentUnitFlat'],2.0,2.0,1.0,2019-01-10T17:10:35.493Z,2018-10-05T04:57:13Z,"{""latitude"": -37.8430561, ""longitude"": 144.94026}",NaN,3207,port melbourne,2019,1,$550.00 per week,NaN,550.0,144.940260,-37.843056,0.364198,69,NaN,0,0.428416,3,NaN,0,0.421795,2.0,0
3,5439,['house'],2.0,3.0,0.0,2019-01-22T00:20:53.707Z,2018-10-05T07:07:14Z,"{""latitude"": -37.846166, ""longitude"": 144.957026}",NaN,3206,albert park,2019,1,$975.00 per week,NaN,975.0,144.957026,-37.846166,0.446261,9,NaN,0,0.461163,21,NaN,0,0.441093,4.0,0
4,5447,['apartmentUnitFlat'],1.0,2.0,1.0,2019-01-17T17:10:07.873Z,2018-10-17T01:02:26Z,"{""latitude"": -37.8408338, ""longitude"": 144.939...",NaN,3207,port melbourne,2019,1,$500.00 per week,NaN,500.0,144.939074,-37.840834,0.366102,69,NaN,0,0.429226,3,NaN,0,0.418889,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80277,333013,['townhouse'],1.0,2.0,1.0,2022-03-21T01:21:52.75Z,2022-03-09T02:28:07Z,"{""latitude"": -36.7365174, ""longitude"": 147.159...",NaN,3698,tawonga south,2022,3,$450 unfurnished or $500 furnished,NaN,NaN,147.159413,-36.736517,NaN,0,NaN,0,NaN,0,NaN,0,NaN,NaN,0
80278,333014,['house'],2.0,3.0,2.0,2022-05-02T03:23:46.94Z,2022-03-23T04:13:37Z,"{""latitude"": -36.73394589999999, ""longitude"": ...",NaN,3698,tawonga south,2022,5,$490 per week,NaN,490.0,147.153535,-36.733946,NaN,0,NaN,0,NaN,0,NaN,0,NaN,NaN,0
80279,333015,['house'],2.0,3.0,1.0,2022-05-13T05:27:18.527Z,2022-05-10T00:57:02Z,"{""latitude"": -36.6861761, ""longitude"": 147.137...",NaN,3697,tawonga,2022,5,$500 per week,NaN,500.0,147.137745,-36.686176,1.565025,1,NaN,0,NaN,0,NaN,0,NaN,0.0,0
80280,333016,['house'],2.0,4.0,0.0,2022-06-22T01:30:54.673Z,2022-05-31T06:16:44Z,"{""latitude"": -36.7329424, ""longitude"": 147.152...",534.0,3698,tawonga south,2022,6,$540 per week,NaN,540.0,147.152703,-36.732942,NaN,0,NaN,0,NaN,0,NaN,0,NaN,NaN,0


In [ ]:
init_house.to_csv("./data/curated/init_house.csv")

In [ ]:
# merge_sc = pd.merge(init_house,shopping_center,how='left',on='postcode')
# merge_sc
# merge_sc["tmp"] = np.sin(merge_sc["stop_latitude"] - merge_sc["latitude"] / 2)**2 + np.cos(merge_sc["latitude"]) * np.cos(merge_sc["stop_latitude"]) * np.sin(merge_sc["stop_longitude"] - merge_vline["longitude"] / 2)**2
# merge_sc["vline_stop_distance"] = 2 * np.arctan2(np.sqrt(merge_sc["tmp"]), np.sqrt(1 - merge_sc["tmp"]))
# vline_stop_min = pd.DataFrame(merge_vline.groupby(["index",'postcode'])["vline_stop_distance"].min())
# vline_stop_count = pd.DataFrame(merge_vline.groupby(["index",'postcode'])["vline_stop_distance"].count())
# init_house = pd.merge(init_house,vline_stop_min,how='left',left_on=['index','postcode'],right_on=['index','postcode'])
# init_house = pd.merge(init_house,vline_stop_count,how='left',left_on=['index','postcode'],right_on=['index','postcode'])
# init_house.rename(columns={"vline_stop_distance_x":"distance of the closest vline stop","vline_stop_distance_y":"count of vline stop"}, inplace=True)

KeyError: 'postcode'

,index,propertyTypes,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,geoLocation,landAreaSqm,postcode,surburb,year,month,price,bond,rent,longitude,latitude,distance of the closest bus stop,count of bus stop,distance of the closest train stop,count of train stop,distance of the closest tram stop,count of tram stop,distance of the closest vline stop,count of vline stop,distance of the closest school,count of school,name
0,4256,['apartmentUnitFlat'],1.0,1.0,0.0,2019-01-09T00:23:18.123Z,2016-02-16T04:33:49Z,"{""latitude"": -37.8341628, ""longitude"": 144.960...",NaN,3205,south melbourne,2019,1,$360.00 per week,NaN,360.0,144.960873,-37.834163,0.436505,10,NaN,0,0.438447,24,NaN,0,0.441059,3.0,0
1,5429,['apartmentUnitFlat'],2.0,4.0,4.0,2019-01-03T22:51:13.063Z,2018-10-02T00:00:38Z,"{""latitude"": -37.8443239, ""longitude"": 144.943...",NaN,3207,port melbourne,2019,1,$1800.00 per week,NaN,1800.0,144.943722,-37.844324,0.360502,69,NaN,0,0.427958,3,NaN,0,0.413926,2.0,0
2,5437,['apartmentUnitFlat'],2.0,2.0,1.0,2019-01-10T17:10:35.493Z,2018-10-05T04:57:13Z,"{""latitude"": -37.8430561, ""longitude"": 144.94026}",NaN,3207,port melbourne,2019,1,$550.00 per week,NaN,550.0,144.940260,-37.843056,0.364198,69,NaN,0,0.428416,3,NaN,0,0.421795,2.0,0
3,5439,['house'],2.0,3.0,0.0,2019-01-22T00:20:53.707Z,2018-10-05T07:07:14Z,"{""latitude"": -37.846166, ""longitude"": 144.957026}",NaN,3206,albert park,2019,1,$975.00 per week,NaN,975.0,144.957026,-37.846166,0.446261,9,NaN,0,0.461163,21,NaN,0,0.441093,4.0,0
4,5447,['apartmentUnitFlat'],1.0,2.0,1.0,2019-01-17T17:10:07.873Z,2018-10-17T01:02:26Z,"{""latitude"": -37.8408338, ""longitude"": 144.939...",NaN,3207,port melbourne,2019,1,$500.00 per week,NaN,500.0,144.939074,-37.840834,0.366102,69,NaN,0,0.429226,3,NaN,0,0.418889,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80277,333013,['townhouse'],1.0,2.0,1.0,2022-03-21T01:21:52.75Z,2022-03-09T02:28:07Z,"{""latitude"": -36.7365174, ""longitude"": 147.159...",NaN,3698,tawonga south,2022,3,$450 unfurnished or $500 furnished,NaN,NaN,147.159413,-36.736517,NaN,0,NaN,0,NaN,0,NaN,0,NaN,NaN,0
80278,333014,['house'],2.0,3.0,2.0,2022-05-02T03:23:46.94Z,2022-03-23T04:13:37Z,"{""latitude"": -36.73394589999999, ""longitude"": ...",NaN,3698,tawonga south,2022,5,$490 per week,NaN,490.0,147.153535,-36.733946,NaN,0,NaN,0,NaN,0,NaN,0,NaN,NaN,0
80279,333015,['house'],2.0,3.0,1.0,2022-05-13T05:27:18.527Z,2022-05-10T00:57:02Z,"{""latitude"": -36.6861761, ""longitude"": 147.137...",NaN,3697,tawonga,2022,5,$500 per week,NaN,500.0,147.137745,-36.686176,1.565025,1,NaN,0,NaN,0,NaN,0,NaN,0.0,0
80280,333016,['house'],2.0,4.0,0.0,2022-06-22T01:30:54.673Z,2022-05-31T06:16:44Z,"{""latitude"": -36.7329424, ""longitude"": 147.152...",534.0,3698,tawonga south,2022,6,$540 per week,NaN,540.0,147.152703,-36.732942,NaN,0,NaN,0,NaN,0,NaN,0,NaN,NaN,0


In [ ]:
# population
income
header = pd.MultiIndex.from_product([['Earner','Median_age','Sum', 'median', 'mean'],['2019','2020','2021','2022']])
# d=([[12,45,67,56],[78,89,45,67],[45,67,89,90],[67,44,56,55]])
# income_earner = income.iloc[:,[0,1,2,3,4,5,6]]
# income_median_age = income.iloc[:,[0,1,2,7,8,9,10]]
# income_sum = income.iloc[:,[0,1,2,11,12,13,14]]
# income_median = income.iloc[:,[0,1,2,15,16,17,18]]
# income_mean = income.iloc[:,[0,1,2,19,20,21,22]]
# income_mean

,SA2,SA2 NAME,Postcode,2019,2020,2021,2022,2019.1,2020.1,2021.1,2022.1,2019.2,2020.2,2021.2,2022.2,2019.3,2020.3,2021.3,2022.3,2019.4,2020.4,2021.4,2022.4
0,201011001,Alfredton,3350.0,8503.447540,9019.555013,9567.571661,10148.885100,42.000000,42.000000,42.000000,42.000000,5.581284e+08,6.112807e+08,6.684159e+08,7.295697e+08,55374.162543,56981.756185,58628.439829,60284.180114,65636.394852,67773.311518,69862.748812,71889.039229
1,201011002,Ballarat,3350.0,7673.610510,7743.676448,7813.745816,7813.745816,47.516546,47.744301,48.030360,48.390075,6.203439e+08,6.428705e+08,6.724473e+08,7.019395e+08,55244.566537,56765.110158,58475.016103,60293.138457,80837.265352,83014.858813,86055.212890,89164.155659
2,201011003,Ballarat - North,3350.0,13887.614446,14253.732963,14617.249150,14617.249150,43.000000,43.000000,43.000000,43.000000,8.662905e+08,9.192677e+08,9.765977e+08,1.027101e+09,52249.545142,53999.918073,56041.662605,57892.944588,62370.887870,64475.677365,66780.534223,68657.186139
3,201011004,Ballarat - South,3350.0,13091.389530,13514.038425,13949.221083,13949.221083,40.006410,40.072527,40.155309,40.259000,6.834107e+08,7.275825e+08,7.741941e+08,8.241449e+08,47167.675189,48689.714936,50364.465559,52008.278179,52199.871922,53834.701955,55495.105372,57237.406910
4,201011005,Buninyong,3350.0,4322.973567,4407.512769,4483.031429,4483.031429,46.255556,46.657079,47.163341,47.541016,2.794734e+08,2.924104e+08,3.049772e+08,3.179211e+08,53825.456103,55531.685800,57244.683920,58912.175016,64645.372883,66339.339696,68023.775165,69741.827133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,217031476,Otway,NaN,2395.620188,2444.316676,2477.970211,2477.970211,50.754808,50.617550,50.446442,50.233280,1.150772e+08,1.211531e+08,1.263859e+08,1.330869e+08,37575.531366,38825.451577,40574.160847,42432.984431,48029.206851,49545.317650,50971.846908,52656.842882
458,217041477,Moyne - East,NaN,4130.790588,4208.920238,4295.801621,4295.801621,46.000000,46.000000,46.000000,46.000000,2.377618e+08,2.549300e+08,2.743324e+08,2.969008e+08,47521.300045,49601.607879,51947.094563,54149.790132,57517.139276,60504.275839,63765.400232,67558.970308
459,217041478,Moyne - West,NaN,5904.175034,5982.515142,6039.823554,6039.823554,48.255319,48.399700,48.580716,48.807833,3.397270e+08,3.560742e+08,3.729009e+08,3.937043e+08,46691.598987,48020.150277,49610.776348,51377.496893,57540.056585,59516.823108,61731.945229,64417.658945
460,217041479,Warrnambool - North,NaN,13412.087251,13692.824973,13954.601692,13954.601692,43.000000,43.000000,43.000000,43.000000,7.745275e+08,8.181133e+08,8.651454e+08,8.902444e+08,50549.583032,52474.220172,54599.175794,56700.443532,57742.626469,59735.611941,61975.387295,62583.220506
